In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import os
import Data

In [2]:
root_path = 'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')
exp_details = exp_details.drop(index=0)  # remove pilot experiment
# exps_to_remove = [20, 23]
# exp_details = exp_details.drop(exp_details[exp_details.Experiment.isin(exps_to_remove)].index)
exp_details

,Experiment,Larvae_present,Colony,P_yellow,C_yellow,P_red,C_red
1,11,yes,A,2,1,1,3
2,12,no,A,2,1,1,3
3,13,no,A,2,1,1,3
4,16,no,B,2,1,1,3
5,17,no,B,2,1,1,3
6,18,yes,A,2,1,1,3
7,19,yes,A,2,1,1,3
8,20,yes,A,2,1,1,3
9,21,no,B,2,1,1,3
10,22,no,B,2,1,1,6


In [3]:
def get_exp_path(exp_num, root_path):
    folderlist = os.listdir(root_path)
    exp_folder = [x for x in folderlist if x.startswith('experiment'+str(exp_num))]
    return root_path + sep + exp_folder[0]

In [4]:
def get_PC_ratios_dict(exp_details,row_idx):
    PC_ratios_dict = {}
    for color in ['yellow', 'red']:
        PC_ratios_dict[color] = [exp_details.loc[row_idx,'P_'+color], exp_details.loc[row_idx,'C_'+color]]
    return PC_ratios_dict

In [5]:
def get_crop_data_dict(exp_num, crop_file_name=r"clean_crops_transparency_corrected.csv"):
    exp_path = get_exp_path(exp_num, root_path)
    crop_data = pd.read_csv(exp_path + sep + crop_file_name, header=[0,1])
    conversion_factors = pd.read_csv(exp_path + sep + r"conversion_factors_by_weight_and_feeding_sum.csv")
    
    crop_data_dict = {}
    for color in ['red','yellow']:
        crop_data_dict[color] = crop_data.xs(color, axis=1, level=1, drop_level=False)/conversion_factors[color][0]
    
    return crop_data_dict

In [6]:
def crop_composition_to_PC_amounts(crop_yellow, crop_red, PC_ratios_dict):
    P_yellow, C_yellow = Data.food_volume_to_PC_amounts(crop_yellow, PC_ratios_dict['yellow'])
    P_red, C_red = Data.food_volume_to_PC_amounts(crop_red, PC_ratios_dict['red'])
    P = P_yellow + P_red
    C = C_yellow + C_red
    return P, C

In [7]:
def get_frame_PC_amounts(crop_data_dict,PC_ratios_dict,frame=-100):
    ants = crop_data_dict['red'].columns.droplevel(1).to_list()
    PC_df = pd.DataFrame(index=ants, columns=['P_mg', 'C_mg'])
    PC_df
    for ant in ants:
        P_ant, C_ant = crop_composition_to_PC_amounts(crop_data_dict['yellow'].iloc[frame][ant]['yellow'], 
                                                      crop_data_dict['red'].iloc[frame][ant]['red'], 
                                                      PC_ratios_dict)
        PC_df.loc[ant]['P_mg'] = P_ant
        PC_df.loc[ant]['C_mg'] = C_ant

    return PC_df

In [12]:
def plot_GF(PC_df, PC_ratios_dict, color, ax):
    
    # food rails
    m = 29
    n = 22
    x_p = PC_ratios_dict['yellow'][0]
    y_p = PC_ratios_dict['yellow'][1]
    x_c = PC_ratios_dict['red'][0]
    y_c = PC_ratios_dict['red'][1]
    ax.plot([0, m*x_p],[0, m*y_p],'k',linewidth=0.8)
    ax.plot([0,n*x_c],[0, n*y_c],'k',linewidth=0.8)
    
    # colony intake line
    ax.plot([0,PC_df['P_mg'].sum()],[0,PC_df['C_mg'].sum()],'--k',linewidth=1)
    
    # ants data
    ax.scatter(PC_df['P_mg'], PC_df['C_mg'], color=color, s=9)
    
    #max_lim = max(PC_df['P_mg'].max(), PC_df['C_mg'].max())*1.1
    max_lim=3
    
    ax.set_xlim([0,max_lim])
    ax.set_ylim([0,max_lim])
    #ax.set_xlabel('Protein [mg]')
    #ax.set_ylabel('Carbohydrates [mg]')
    ax.set_aspect('equal', 'box')
    #ax.tight_layout()

In [13]:
larvae_present = exp_details.Experiment[exp_details.Larvae_present == 'yes'].to_list()
no_larvae = exp_details.Experiment[(exp_details.Colony == 'B') & (exp_details.C_red==3)].to_list()
extra_sugar = exp_details.Experiment[exp_details.C_red==6].to_list()

In [14]:
%matplotlib qt
fig, ax = plt.subplots(2,4)
counter = 0
for exp_num in larvae_present:
    exp_path = get_exp_path(exp_num, root_path)
    exp_idx = exp_details.index[exp_details.Experiment==exp_num][0]
    PC_ratios_dict = get_PC_ratios_dict(exp_details,exp_idx)
    crop_data_dict = get_crop_data_dict(exp_num)
    PC_df = get_frame_PC_amounts(crop_data_dict,PC_ratios_dict)
    plot_GF(PC_df, PC_ratios_dict, 'b', ax[0,counter])
    counter += 1

counter = 0
for exp_num in no_larvae:
    exp_path = get_exp_path(exp_num, root_path)
    exp_idx = exp_details.index[exp_details.Experiment==exp_num][0]
    PC_ratios_dict = get_PC_ratios_dict(exp_details,exp_idx)
    crop_data_dict = get_crop_data_dict(exp_num)
    PC_df = get_frame_PC_amounts(crop_data_dict,PC_ratios_dict)
    plot_GF(PC_df, PC_ratios_dict, 'm', ax[1,counter])
    counter += 1

fig.supxlabel('P [mg]')
ax[0,0].set_ylabel('C [mg]')
ax[1,0].set_ylabel('C [mg]')

Text(0, 0.5, 'C [mg]')

In [15]:
fig, ax = plt.subplots()
plot_GF(PC_df, PC_ratios_dict, 'b', ax)

In [ ]:
ants = crop_data_dict['red'].columns.droplevel(1).to_list()
PC_df = pd.DataFrame(index=ants, columns=['P_mg', 'C_mg'])
PC_df
for ant in ants:
    P_ant, C_ant = crop_composition_to_PC_amounts(crop_data_dict['yellow'].iloc[-100][ant]['yellow'], 
                                                  crop_data_dict['red'].iloc[-100][ant]['red'], 
                                                  PC_ratios_dict)
    PC_df.loc[ant]['P_mg'] = P_ant
    PC_df.loc[ant]['C_mg'] = C_ant

PC_df

In [ ]:
plt.scatter(PC_df['P_mg'],PC_df['C_mg'])

In [ ]:
PC_ratios_dict['yellow']

In [ ]:
crop_data_dict['yellow'].iloc[-100]


In [ ]:
ant

In [ ]:
ants = crop_data_dict['red'].columns.droplevel(1).to_list()
ants

In [ ]:
crop_data_dict['red']